In [1]:
get_ipython().system(u'ls -lL /tmp/')

total 0
drwxrwx---. 4 1000840000 wscommon 49 Jun 26 16:32 1000840000


In [2]:
username = 'cpadmin'
api_key = 'iwaj8savPEZitY6vXJOjpgq5QlY6RF0PJpjEeLAA'
url = 'https://cpd-cpd-instance.apps.ai.toropsp.com/'

wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '5.0'
}

In [3]:
!pip install -U ibm-watson-machine-learning 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm_watson_machine_learning 1.0.357
    Uninstalling ibm_watson_machine_learning-1.0.357:
      Successfully uninstalled ibm_watson_machine_learning-1.0.357


In [4]:
from ibm_watson_machine_learning import APIClient 
client = APIClient(wml_credentials) 
space_id ='274032d5-1ec4-48b8-b6de-77da8ac0f17c' 
client.set.default_space(space_id) 

'SUCCESS'

In [5]:
def Granite3B_fp32_ONNX_Inference():

    # ---------------------------------------------------------------- LIBRARY INSTALLATION ----------------------------------------------------------------
    # To install some of the required packages 
    import subprocess  
    subprocess.run(['conda', 'install', 'onnxruntime', '-c', 'https://ausgsa.ibm.com:7191/gsa/ausgsa/projects/o/open-ce/conda/Open-CE-r1.11/1.11.0/opence-p10/']) 
    subprocess.run(['pip', 'install', 'optimum']) 
    subprocess.run(['pip', 'install', 'transformers']) 
    subprocess.run(['pip', 'install', 'huggingface_hub'])
    subprocess.run(['pip', 'install', 'boto3==1.26.90'])
    subprocess.run(['pip', 'install', 'botocore==1.26.90'])
    subprocess.run(['conda', 'install', 'pyyaml', '-c', 'defaults'])
    subprocess.run(['pip', 'install', 'accelerate'])
    subprocess.run(['pip', 'install', '-U', 'ibm-watson-machine-learning']) 
    
    # ---------------------------------------------------------------- API CLIENT ----------------------------------------------------------------
    # In order to use the Cloud Pak for Data (CP4D) API service inside the deployment, you need to create an API client.
    username = 'cpadmin'
    api_key = 'iwaj8savPEZitY6vXJOjpgq5QlY6RF0PJpjEeLAA'
    url = 'https://cpd-cpd-instance.apps.ai.toropsp.com/'
    
    wml_credentials = {
        "username": username,
        "apikey": api_key,
        "url": url,
        "instance_id": 'openshift',
        "version": '5.0'
    }

    from ibm_watson_machine_learning import APIClient 
    client = APIClient(wml_credentials) 
    space_id ='274032d5-1ec4-48b8-b6de-77da8ac0f17c' 
    client.set.default_space(space_id) 
    
    subprocess.run(['mkdir', '/tmp/granite-3b']) 

    # ---------------------------------------------------------------- Granite-3B config.json creation ----------------------------------------------------------------
    import os
    import json
    
    # Define the directory and file path
    directory = "/tmp/granite-3b"
    file_path = os.path.join(directory, "config.json")
    
    # Define the JSON content
    config_content = {
      "activation_function": "gelu",
      "architectures": [
        "GPTBigCodeForCausalLM"
      ],
      "attention_softmax_in_fp32": True,
      "attn_pdrop": 0.1,
      "bos_token_id": 0,
      "embd_pdrop": 0.1,
      "eos_token_id": 0,
      "initializer_range": 0.02,
      "layer_norm_epsilon": 1e-05,
      "model_type": "gpt_bigcode",
      "multi_query": True,
      "n_embd": 3072,
      "n_head": 32,
      "n_inner": 12288,
      "n_layer": 32,
      "n_positions": 2048,
      "pad_token_id": 0,
      "resid_pdrop": 0.1,
      "scale_attention_softmax_in_fp32": True,
      "scale_attn_weights": True,
      "torch_dtype": "float32",
      "transformers_version": "4.31.0",
      "use_cache": True,
      "vocab_size": 50304
    }
    
    # Write the JSON content to the file
    with open(file_path, "w") as json_file:
        json.dump(config_content, json_file, indent=2)
    
    print(f"File {file_path} created successfully.")

    # ---------------------------------------------------------------- Granite-3B fp32 PyTorch download ----------------------------------------------------------------
    import boto3
    from botocore.client import Config
    #from botocore import botocore.utils
    
    s3_config = {
      "s3_user": "f25e4fb594da49cf93009caa617dc254",
      "s3_pass": "e43ae432d430d9dd4ea2ebb28f400f6c5d5b6134820720a3",
      "s3_host": "http://s3.us-east.cloud-object-storage.appdomain.cloud",
      "s3_bucket": "inter-ckpts",
    }
    
    s3_client = boto3.session.Session().resource(
      service_name="s3",
      endpoint_url=s3_config["s3_host"],
      aws_access_key_id=s3_config["s3_user"],
      aws_secret_access_key=s3_config["s3_pass"],
      config=Config(signature_version="s3v4"),
    )
    bucket = s3_client.Bucket(s3_config["s3_bucket"])
    
    for bucket_object in bucket.objects.all():
        print(bucket_object)
    
    bucket.download_file("granite-3b-base-v2/step_75000_ckpt/config.json","/tmp/granite-3b/generation_config.json" )
    bucket.download_file("granite-3b-base-v2/step_75000_ckpt/pytorch_model-00001-of-00002.bin","/tmp/granite-3b/pytorch_model-00001-of-00002.bin" )
    bucket.download_file("granite-3b-base-v2/step_75000_ckpt/pytorch_model-00002-of-00002.bin","/tmp/granite-3b/pytorch_model-00002-of-00002.bin" )
    bucket.download_file("granite-3b-base-v2/step_75000_ckpt/pytorch_model.bin.index.json","/tmp/granite-3b/pytorch_model.bin.index.json" )
    bucket.download_file("granite-3b-base-v2/step_75000_ckpt/special_tokens_map.json","/tmp/granite-3b/special_tokens_map.json" )
    bucket.download_file("granite-3b-base-v2/step_75000_ckpt/tokenizer.json","/tmp/granite-3b/tokenizer.json" )
    bucket.download_file("granite-3b-base-v2/step_75000_ckpt/tokenizer_config.json","/tmp/granite-3b/tokenizer_config.json" )

    # ---------------------------------------------------------------- MODEL RETRIEVAL ----------------------------------------------------------------
    subprocess.run(['optimum-cli', 'export', 'onnx', '--model', '/tmp/granite-3b', '/tmp/granite-3b-onnx', '--task', 'text-generation-with-past'])

    # ---------------------------------------------------------------- LIBRARY IMPORTS ----------------------------------------------------------------
    
    from transformers import AutoTokenizer
    from optimum.onnxruntime import ORTModelForCausalLM
    import time
    import torch
    import onnxruntime
    from onnxruntime import ExecutionMode

    def inference(input, model_id="/tmp/granite-3b-onnx"):

        # Define the question extraction function
        def extract_question(input_data):
            return input_data["input_data"][0]["values"][0]
            #return input_data["input_data"][0]["values"][0][0]
            #return [item[0] for item in input["input_data"][0]["values"]]
        
        # Extract the question from input_data
        questions = extract_question(input)
        
        # Set the random seed for reproducibility
        torch.manual_seed(16)

        # Set number of threads
        options = onnxruntime.SessionOptions()
        options.intra_op_num_threads = 12
        options.inter_op_num_threads = 12
    
        tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")
        tokenizer.pad_token = tokenizer.eos_token
    
        model = ORTModelForCausalLM.from_pretrained(model_id, use_cache=True, use_merged=False, use_io_binding=False, session_options=options)
    
        # Tokenize questions and calculate the total number of tokens
        question_tokens = sum(len(tokenizer.encode(question)) for question in questions)
    
        # Tokenize all questions at once
        inputs = tokenizer(questions, return_tensors="pt", padding=True, truncation=True)
    
        # Generate responses
        res = model.generate(**inputs, do_sample=True, min_new_tokens=128, max_new_tokens=128, temperature=0.7, top_k=40, top_p=0.95)
    
        # Decode and print the generated responses
        outputs = tokenizer.batch_decode(res, skip_special_tokens=True)
        """
        for i, out in enumerate(outputs):
            print(f"Result for Question {i + 1}: {out}")
            print(len(tokenizer.encode(out)))
        
        # Calculate total tokens in the generated responses
        total_tokens = sum(len(tokenizer.encode(out)) for out in outputs)
        # Subtract the question tokens from total tokens to get tokens generated for answers
        total_tokens -= question_tokens
        """
        prediction_response = { 
            'predictions': [{
                             'values': [outputs] 
                             }] 

        }
        return prediction_response
    
    return inference

In [6]:
# Creating the python fuction in the deployment space with specific software specification 
pyfunc_swspec_id = client.software_specifications.get_uid_by_name("runtime-24.1-py3.11") 

meta_data = { 
    client.repository.FunctionMetaNames.NAME: 'Granite-3B FP32 ONNX', 
    client.repository.FunctionMetaNames.DESCRIPTION: 'Granite-3B fp32 ONNX Inferencing', 
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id 
} 

function_details = client.repository.store_function(meta_props=meta_data, function=Granite3B_fp32_ONNX_Inference) 

In [12]:
function_uid = client.repository.get_function_uid(function_details) 

meta_props = { 
   client.deployments.ConfigurationMetaNames.NAME: "Granite-3B FP32 ONNX deployment 12 threads dp6", 
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "GRANITE-3B FP32 ONNX WITH 64 GB MEMORY AND 16 vCPU", 
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'XL'}, 
   client.deployments.ConfigurationMetaNames.ONLINE: {   } 
} 

deployment_details = client.deployments.create(function_uid, meta_props=meta_props) 

print(deployment_details) 

deployment_id = client.deployments.get_uid(deployment_details) 



#######################################################################################

Synchronous deployment creation for uid: '9f6aa966-34d3-4218-9fc2-0f452bc7ea3d' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..................................................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='cfc57af1-0d15-4129-bb0e-5376590ff733'
------------------------------------------------------------------------------------------------


{'entity': {'asset': {'id': '9f6aa966-34d3-4218-9fc2-0f452bc7ea3d'}, 'custom': {}, 'deployed_asset_type': 'function', 'description': 'GRANITE-3B FP32 ONNX WITH 64 GB MEMORY AND 16 vCPU', 'hardware_spec': {'id': 'd0aa1ae8-a889-42e2-a099-041b604b9289', 